In [6]:
import numpy as np, geopandas as gpd, rasterio
from rasterio.mask import mask
import pandas as pd

NDVI_1KM = "../data/processed/ndvi_clean/ndvi_summer_2024_mean_1km_match_lst.tif"
LST_1KM  = "../data/processed/lst_clean/lst_summer_2024_mean_sinusoidal.tif"
NEIGH    = "../data/raw/neighbourhood-boundaries.geojson"
OUT_CSV  = "../data/processed/neighbourhood_stats/ndvi_lst_neighbourhood_stats_2024.csv"

# Load rasters + crs/transform
ndvi_src = rasterio.open(NDVI_1KM)
lst_src  = rasterio.open(LST_1KM)
assert ndvi_src.crs == lst_src.crs, "CRS mismatch; ensure both are sinusoidal 1 km"
neigh = gpd.read_file(NEIGH).to_crs(ndvi_src.crs)

rows = []
name_col = next((c for c in ["NAME","Name","neighbourhood","neighbourhood","AREA_NAME","AREA_NAME_EN"] if c in neigh.columns), neigh.columns[0])

for i, geom in enumerate(neigh.geometry):
    # NDVI stats
    ndvi_clip, _ = mask(ndvi_src, [geom], crop=True)
    ndvi = ndvi_clip[0]
    ndvi_valid = np.isfinite(ndvi)
    ndvi_mean = float(np.nanmean(ndvi)) if ndvi_valid.any() else np.nan
    ndvi_pct_valid = float(ndvi_valid.mean()*100)

    # LST stats
    lst_clip, _ = mask(lst_src, [geom], crop=True)
    lst = lst_clip[0]
    lst_valid = np.isfinite(lst)
    lst_mean = float(np.nanmean(lst)) if lst_valid.any() else np.nan
    lst_pct_valid = float(lst_valid.mean()*100)

    rows.append({
        "neighbourhood": str(neigh.iloc[i][name_col]),
        "ndvi_mean_1km": ndvi_mean,
        "ndvi_pct_valid": ndvi_pct_valid,
        "lst_mean_c": lst_mean,
        "lst_pct_valid": lst_pct_valid
    })

pd.DataFrame(rows).to_csv(OUT_CSV, index=False)
ndvi_src.close(); lst_src.close()
print("✅ wrote", OUT_CSV)


✅ wrote ../data/processed/neighbourhood_stats/ndvi_lst_neighbourhood_stats_2024.csv
